In [1]:
import numpy as np
import pandas as pd

In [14]:
data = pd.DataFrame.from_records(np.load('results/mal_cfr_data.npy', allow_pickle=True))
data

,game_tag,sampler,seed,mode,row_alg,col_alg,value
0,tiny_bridge,null,2.0,shuffled,CFR,CFR,20.20380
1,tiny_bridge,null,2.0,shuffled,CFR,CFR+,20.10390
2,tiny_bridge,null,2.0,shuffled,CFR,A-CFR,16.37950
3,tiny_bridge,null,2.0,shuffled,CFR,A-CFR+,16.57310
4,tiny_bridge,null,2.0,shuffled,CFR,PGPI(1),6.81680
...,...,...,...,...,...,...,...
1773,leduc,null,NaN,fixed,BR,A-CFR,1.76323
1774,leduc,null,NaN,fixed,BR,A-CFR+,1.39958
1775,leduc,null,NaN,fixed,BR,PGPI(1),1.35053
1776,leduc,null,NaN,fixed,BR,PGPI(100/sqrt(T)),1.41062


In [15]:
games = data.game_tag.unique()
games

array(['tiny_bridge', 'goofspiel', 'kuhn_3p', 'random_goofspiel', 'leduc'],
      dtype=object)

In [17]:
modes = data['mode'].unique()
modes

array(['shuffled', 'sim', 'fixed'], dtype=object)

In [119]:
def det_table(game, mode):
    assert mode == 'fixed' or mode == 'sim'
    df = data.query(f'game_tag == "{game}" & mode == "{mode}"')
    return df.pivot(index='row_alg', columns='col_alg', values='value')

def without_br_row(df):
    return df.query('row_alg != "BR"')

def with_avg_row_col(df):
    col_avgs = df.mean(axis=0).to_frame('avg').transpose()
    row_avgs = df.mean(axis=1)
    row_avgs = pd.concat([row_avgs, pd.Series({'avg': row_avgs.mean()})], axis=0)
    row_avgs.name = 'avg'
    df = pd.concat([df, col_avgs], axis=0)
    return pd.concat([df, row_avgs], axis=1)

In [120]:
print((with_avg_row_col(det_table('kuhn_3p', 'fixed')) * 1000).to_latex(float_format="{:0.1f}".format, index_names=False))

\begin{tabular}{lrrrrrrrr}
\toprule
{} &  A-CFR &  A-CFR+ &   CFR &  CFR+ &  PGPI(1) &  PGPI(100/sqrt(T)) &    PI &   avg \\
\midrule
A-CFR             &    0.0 &    -0.2 & -11.6 &  -4.0 &     74.4 &               74.4 & 315.5 &  64.1 \\
A-CFR+            &   46.7 &     0.0 &  60.1 &  -4.1 &     85.9 &               85.9 & 309.3 &  83.4 \\
BR                &  196.1 &   107.0 & 117.5 &   8.1 &    139.7 &              139.7 & 692.4 & 200.1 \\
CFR               &   58.3 &    54.8 &  -0.0 &   0.1 &    102.2 &              102.2 & 393.8 & 101.6 \\
CFR+              &  147.8 &    54.3 &  91.8 &  -0.0 &    138.4 &              138.4 & 386.3 & 136.7 \\
PGPI(1)           &    1.7 &    -2.6 &  -4.2 &  -6.7 &      0.0 &                0.0 & 332.6 &  45.8 \\
PGPI(100/sqrt(T)) &    1.7 &    -2.6 &  -4.2 &  -6.7 &      0.0 &                0.0 & 332.6 &  45.8 \\
PI                &  195.8 &   106.6 & 117.2 &   7.5 &    139.7 &              139.7 &   0.0 & 100.9 \\
avg               &   81.0 &    39